In [1]:
from usgs import api

# Set the EarthExplorer catalog
node = 'EE'# this indicates earth explorer website

# Set the Hyperion and Landsat 8 dataset
hyperion_dataset = 'EO1_HYP_PUB'
landsat8_dataset = 'LANDSAT_8'

# Set the scene ids
hyperion_scene_id = 'EO1H1820422014302110K2_SG1_01'
landsat8_scene_id = 'LC80290462015135LGN00'

# Submit requests to USGS servers
api.metadata(hyperion_dataset, node, [hyperion_scene_id])
api.metadata(landsat8_dataset, node, [landsat8_scene_id])

usgs.api.metadata(dataset, node, entityids, extended=False, api_key=None)

AttributeError: module 'usgs.api' has no attribute 'metadata'

## Sentinelsat and Copernicus

In [2]:
# https://towardsdatascience.com/satellite-imagery-access-and-analysis-in-python-jupyter-notebooks-387971ece84b

In [6]:
# https://sentinelsat.readthedocs.io/en/stable/

In [3]:
from sentinelsat import SentinelAPI

user = 'COPERNICUS_USERNAME' 
password = 'COPERNICUS_PASSWORD' 
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [ ]:
import geopandas as gpd
import folium 

# santa monica bounds file named nReserve
nReserve = gpd.read_file('/Users/sra/Files/brainstation_2023_ds_capstone/brainstation_2023_ds_capstone/01_capstone_data/shapefiles/santa_monica_bounds/sm_bounds.geojson')

# empty base map in Folium centered around the bounding box
m = folium.Map([34.08483,-118.70617], zoom_start=12)
folium.GeoJson(nReserve).add_to(m)
m

In [ ]:
from shapely.geometry import MultiPolygon, Polygon

footprint = None
for i in nReserve['geometry']:
    footprint = i

In [ ]:
products = api.query(footprint,
                     #date = ('20190601', '20190626'),
                     date = ('20220101', '20221231'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,15)
                    )

In [ ]:
products

In [ ]:
products_gdf = api.to_geodataframe(products)
products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
products_gdf_sorted
products_gdf

In [4]:
# api.download('becdf74b-fb47-4010-84f1-2c271a501266')

In [ ]:
import rasterio

R10 = '/Users/sra/Files/brainstation_2023_ds_capstone/brainstation_2023_ds_capstone/01_capstone_data/notebooks/S2A_MSIL2A_20221224T183801_N0509_R027_T11SLT_20221224T213852.SAFE/GRANULE/L2A_T11SLT_A039206_20221224T184116/IMG_DATA/R10m'
b4 = rio.open(R10+'/T11SLT_20221224T183801_B04_10m.jp2')
b3 = rio.open(R10+'/T11SLT_20221224T183801_B03_10m.jp2')
b2 = rio.open(R10+'/T11SLT_20221224T183801_B02_10m.jp2')


# Create an RGB image 
with rio.open('RGB.tiff','w',driver='Gtiff', width=b4.width, height=b4.height, 
              count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
    rgb.write(b2.read(1),1) 
    rgb.write(b3.read(1),2) 
    rgb.write(b4.read(1),3) 
    rgb.close()

## another one

In [ ]:
# define filenames

# raster in
rasin='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip.tif'
# CRS:
# Coordinate Reference System (CRS)
# Name				EPSG:26911 - NAD83 / UTM zone 11N
# Units				meters
# Method			Universal Transverse Mercator (UTM)
# Celestial body	Earth
# Reference			Static (relies on a datum which is plate-fixed)


# shapefile in
shpin='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_le2018_merge_shp.shp'
# I reprojected the shapefile to the same crs as the raster above in QGIS
# Coordinate Reference System (CRS)

# Name
# EPSG:26911 - NAD83 / UTM zone 11N
# Units
# meters
# Method
# Universal Transverse Mercator (UTM)
# Celestial body
# Earth
# Reference
# Static (relies on a datum which is plate-fixed)


# raster out
rasout='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_sm_fire.tif'

#                 out     in
result=gdal.Warp(rasout,rasin,cutlineDSName=shpin)

iface.addRasterLayer(rasout)

## https://opensourceoptions.com/blog/how-to-clip-a-raster-to-a-polygon-or-an-extent-with-python-extract-by-mask/

In [ ]:
from osgeo import gdal

# fn_in = r"/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip.tif"
rast_in='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/m_3411849_se_11_060_20180722.tif'

# fn_clip = '/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_sm_fire.tif'


# fn_poly = r"path/to/polygon.gpkg"

# # gdal.Warp(fn_clip, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=True)

# # from osgeo import gdal
# # fn_in = r"..\..\course-gdal-python\data\input\USGS_one_meter_x64y486_ID_FEMAHQ_2018.tif"
# fn_in=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip.tif'

# # fn_poly = r"..\..\course-gdal-python\data\input\polygon.gpkg"
# fn_poly=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_le2018_merge.geojson'

# # fn_clip = '../data/output/clip_polygon.tif'
# fn_clip='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip_fire.tif'


# ortho_fire=gdal.Warp(fn_clip, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=True)
# ortho_fire
# # gdal.Warp(fn_clip2, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=False)
# # where = "ID = '1'"
# # gdal.Warp(fn_clip_where1, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=True, cutlineWhere=where)
# # where = "ID = '2'"
# # gdal.Warp(fn_clip_where2, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=True, cutlineWhere=where)

## rasterio

In [ ]:
# plot the data

show((data,4),cmap='terrain')

In [ ]:
# open raster in read mode
data=rasterio.open(fp)

In [5]:
type(data)

NameError: name 'data' is not defined

In [ ]:
# try to insert geometry into a geodataframe

polygon=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/santa_monica_water_bounds/sm_bounds_water_epsg26911_nad83_utm11n.geojson'

geo=gpd.GeoDataFrame({'geometry':polygon},index=[0],crs=from_epsg(4326))

In [ ]:
# try to re-project into the same coordinate system as the raster data

geo = geo.to_crs(crs=data.crs.data)

In [ ]:
# get coordinates of the geometry in a format that rasterio wants them
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [7]:
coords = getFeatures(geo)
print(coords)

NameError: name 'getFeatures' is not defined

In [8]:
# Now we are ready to clip the raster with the polygon 
# using the coords variable that we just created. 
# Clipping the raster can be done easily with the mask function 
# that we imported in the beginning from rasterio, and specifying clip=True.

out_img, out_transform = mask(raster=data, shapes=coords, crop=True)

NameError: name 'mask' is not defined

## clip raster to polygon

In [ ]:
import shapely

In [ ]:
shapely.geos.geos_version

In [ ]:
# first, merge perimeters
# https://stackoverflow.com/questions/61035170/merging-polygon-shapefiles-in-python

# this was done in QGIS for now. View merged perimeters, 
# to confirm only one polygon in the dataset
sm_perim_merge=gpd.read_file('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_le2018_merge.geojson')
sm_perim_merge

In [ ]:
# get coordinates from geometry of sm_perim_merge

sm_perim_merge['geometry']

In [ ]:
import gdal

gdal.UseExceptions()

In [ ]:
import cv2
from PIL import Image

# filepaths
target_tiff_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/img_patch_fire2.tif'
output_location_fire='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches'

# load the image using Pillow
with Image.open('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/img_patch_fire2.tif') as img:
    # convert the image to a format that OpenCV can handle (e.g., JPEG)
    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

# display the image using OpenCV
cv2.imshow('image', img_cv)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Set the directory containing the images
directory = "/Users/sra/temp"

# Set the desired size of the square image
desired_size = 128

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        # Load the image
        image_path = os.path.join(directory, filename)
        image = Image.open(image_path)
        
        # Get the size of the image
        width, height = image.size
        
        # Check if the image is already square
        if width == height:
            continue
        
        # Calculate the new size of the image
        if width < height:
            new_width = desired_size
            new_height = int(desired_size * height / width)
        else:
            new_height = desired_size
            new_width = int(desired_size * width / height)
        
        # Resize the image
        new_image = image.resize((new_width, new_height))
        
        # Create a new blank square image
        new_square_image = Image.new("RGB", (desired_size, desired_size), (255, 255, 255))
        
        # Paste the resized image onto the center of the square image
        left = (desired_size - new_width) // 2
        top = (desired_size - new_height) // 2
        new_square_image.paste(new_image, (left, top))
        
        # Save the new square image
        new_image_path = os.path.join(directory, f"{os.path.splitext(filename)[0]}_square.jpg")
        new_square_image.save(new_image_path)


In [ ]:
import os
import cv2

source_folder = '/Users/sra/temp'
destination_folder = '/Users/sra/temp2'

for filename in os.listdir(source_folder):
    if filename.endswith('.jpg'):
        img_path = os.path.join(source_folder, filename)
        img = cv2.imread(img_path)
        h, w, _ = img.shape
        if h != w:
            # stretch the image to make it square
            img = cv2.resize(img, (128, 128))
        cv2.imwrite(os.path.join(destination_folder, filename), img)


In [ ]:
# # define input and output directories
# input_dir = "input_folder/"
# output_dir = "output_folder/"

# # loop through all files in input directory
# for filename in os.listdir(input_dir):
#     # check if the file is a TIFF image
#     if filename.endswith(".tif"):
#         # open the image using PIL
#         with Image.open(input_dir + filename) as img:
#             width, height = img.size
#             # check if the image is already square
#             if width == height:
#                 img.save(output_dir + filename)
#             else:
#                 # stretch the image to be square (128x128 pixels)
#                 if width > height:
#                     new_height = 128
#                     new_width = int(width * new_height / height)
#                     img = img.resize((new_width, new_height))
#                     left = (new_width - 128) / 2
#                     top = 0
#                     right = (new_width + 128) / 2
#                     bottom = 128
#                     img = img.crop((left, top, right, bottom))
#                     img.save(output_dir + filename)
#                 else:
#                     new_width = 128
#                     new_height = int(height * new_width / width)
#                     img = img.resize((new_width, new_height))
#                     left = 0
#                     top = (new_height - 128) / 2
#                     right = 128
#                     bottom = (new_height + 128) / 2
#                     img = img.crop((left, top, right, bottom))
#                     img.save(output_dir + filename)

In [ ]:
def separateNonSquareImages(input_path, output_path):
    '''
    
    Separates non-square images from square images from a directory
    and sends them to a separate user-defined folder.
    
    Square images are those whose width and height are the same.
    For example, a 128x128-pixel image is square. 29x128-pixel
    image is not square.
    
    ----
    Inputs
    
    >input_path
    directory location of where the images (both square and non-)
    are located
    
    >output_path
    new destination directory for the non-square images
    '''
    
    os.makedirs(output_path, exist_ok=True)
    for filename in os.listdir(input_path):
        image_path = os.path.join(input_path, filename)
        if os.path.isfile(input_path):
            with Image.open(input_path) as im:
                width, height = im.size
                if width != height:
                    new_filepath = os.path.join(output_path, filename)
                    os.rename(input_path, output_path)

In [ ]:
input_path='/Users/sra/temp'
output_path='/Users/sra/temp3'

for filename in os.listdir(input_path):
    print(filename)
    input_path = os.path.join(inputPath, file_name)
            output_path = os.path.join(outputPath, 
                                       file_name.replace(oldExtension,
                                                         newExtension))

    try:
        img=Image.open(filename)
        img_size=img.size
        if img_size[0]!=img_size[1]:
            print('not the same size')
        else:
            print("it's the same size")
    except:
        pass

In [ ]:
import os.path

filename = os.path.join('/Users/sra/temp/patch_nofire_857.jpg')
img = Image.open(filename)
img_size=img.size
if img_size[0]!=img_size[1]:
    print('not the same size')
else:
    print("it's the same size")

In [ ]:
# inputPath_nofire_city='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/patch_nofire_city'
# inputPath_nofire_farm='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/patch_nofire_farm'
# inputPath_fire_fire1='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/patch_fire_fire1'
# inputPath_fire_fire2='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/patches/patch_fire_fire2'

# inputPaths=[inputPath_nofire_city,
#             inputPath_nofire_farm,
#             inputPath_fire_fire1,
#             inputPath_fire_fire2]

In [ ]:
# for inp in (inputPaths):
#     fileDeleter(source=inp)